# 영어-Claude-v2 Model: Retrieval Augmented Question & Answering with Amazon Bedrock using LangChain

---
### 중요
- 이 노트북은 Anthropic 의 Claude-v2 모델 접근 가능한 분만 실행 가능합니다. 
- 접근이 안되시는 분은 노트북의 코드와 결과 만을 확인 하시면 좋겠습니다.
- 만일 실행시에는 **"과금"** 이 발생이 되는 부분 유념 해주시기 바랍니다.

---

# 0. 기본 지식

### 문맥
우리는 Bedrock에서 사용 가능한 모델을 활용하고 교육 중에 배운 지식을 바탕으로 질문하고 수동 컨텍스트를 제공하는 접근 방식을 탐색했습니다. 이러한 접근 방식은 짧은 문서나 단일 톤 애플리케이션에서는 작동하지만, 모델에 전송된 프롬프트에 모두 들어맞을 수 없는 대규모 기업 문서가 있을 수 있는 기업 수준의 질문 응답으로 확장되지 않습니다.

### 패턴
RAG(Retreival Augmented Generation)라는 아키텍처를 구현하여 이 프로세스를 개선할 수 있습니다. RAG는 ​​언어 모델 외부(비모수적)에서 데이터를 검색하고 검색된 관련 데이터를 컨텍스트에 추가하여 프롬프트를 강화합니다.

이 노트에서는 질문 응답 패턴에 접근하여 문서를 찾고 활용하여 사용자 질문에 대한 답변을 제공하는 방법을 설명합니다.

### 챌린지
- 토큰 한도를 초과하는 대용량 문서 관리 방법
- 질문과 관련된 문서를 찾는 방법

### 제안
위의 과제에 대해 이 노트는 다음과 같은 전략을 제안합니다.
#### 도큐먼트 준비
![임베딩](./images/Embeddings_lang.png)

질문에 답하려면 먼저 문서를 처리하고 문서 저장소 인덱스에 저장해야 합니다.
- 문서를 로드
- 더 작은 덩어리(chunk)로 처리하고 분할합니다.
- Amazon Bedrock Titan Embeddings 모델을 사용하여 각 청크의 수치 벡터 표현 생성
- 청크와 해당 임베딩을 사용하여 인덱스 생성
#### Ask Question
![질문](./images/Chatbot_lang.png)

문서 색인이 준비되면 질문할 준비가 된 것이며, 질문한 질문에 따라 관련 문서를 가져옵니다. 다음 단계가 실행됩니다.
- 입력 질문의 임베딩 생성
- 질문 임베딩과 인덱스의 임베딩을 비교합니다.
- (상위 N) 관련 문서 청크를 가져옵니다.
- 프롬프트에서 컨텍스트의 일부로 해당 청크를 추가합니다.
- Amazon Bedrock의 모델에 프롬프트를 보냅니다.
- 검색된 문서를 기반으로 상황에 맞는 답변을 얻습니다.

## 사용 사례
#### 데이터세트
이 아키텍처 패턴을 설명하기 위해 "Samsung Knox 문서를 사용합니다. 이 문서에서는 다음과 같은 주제를 설명합니다.
- 삼성전자 Knox 내용
- 삼성전자 Knox 보안 솔루션

#### 페르소나
Samsung knox 문서 내용을 상세히 이해하지 못하는 일반인의 페르소나를 가정해 보겠습니다.
모델은 문서를 바탕으로 쉬운 언어로 답변하려고 노력할 것입니다.

### 구현
RAG 접근 방식을 따르기 위해 이 노트북은 RAG와 같은 패턴을 효율적으로 구축할 수 있는 다양한 서비스 및 도구와 통합된 LangChain 프레임워크를 사용하고 있습니다. 우리는 다음 도구를 사용할 것입니다:

- **LLM(대형 언어 모델)**: Amazon Bedrock을 통해 사용 가능한 Anthropic Claude V2

  이 모델은 문서 덩어리(chunk)를 이해하고 인간 친화적인 방식으로 답변을 제공하는 데 사용됩니다.
- **임베딩 모델**: Amazon Bedrock을 통해 사용 가능한 Amazon Titan 임베딩

  이 모델은 텍스트 문서의 수치 표현을 생성하는 데 사용됩니다.
- **문서 로더**: LangChain을 통해 사용 가능한 PDF 로더

  이는 소스에서 문서를 로드할 수 있는 로더입니다. 이 노트북을 위해 로컬 경로에서 샘플 파일을 로드합니다. 이는 기업 내부 시스템에서 문서를 로드하는 로더로 쉽게 대체될 수 있습니다.

- **Vector Store**: LangChain을 통해 FAISS 사용

  이 노트북에서는 이 메모리 내 벡터 저장소를 사용하여 임베딩과 문서를 모두 저장합니다. 기업 환경에서는 이는 AWS OpenSearch, pgVector가 포함된 RDS Postgres, ChromaDB, Pinecone 또는 Weaviate와 같은 영구 저장소로 대체될 수 있습니다.
- **색인**: 벡터인덱스

  인덱스는 입력 임베딩과 문서 임베딩을 비교하여 관련 문서를 찾는 데 도움이 됩니다.
- **래퍼**: 인덱스, 벡터 저장소, 임베딩 모델 및 LLM을 래핑하여 사용자의 논리를 추상화합니다.

### 설정

이 노트북의 나머지 부분을 실행하기 전에 아래 셀을 실행하여 (필요한 라이브러리가 설치되어 있는지 확인하고) Bedrock에 연결해야 합니다.

이 노트북에는 몇 가지 추가 종속성도 필요합니다.

- [FAISS](https://github.com/facebookresearch/faiss), 벡터 임베딩 저장
- [PyPDF](https://pypi.org/project/pypdf/), PDF 파일 처리용

# 1. Bedrock Client 생성

In [2]:
! pip list | grep langchain
! pip list | grep opensearch

langchain                            0.0.312
opensearch-py                        2.3.2


In [3]:
%load_ext autoreload
%autoreload 2

import sys, os
module_path = ".."
sys.path.append(os.path.abspath(module_path))

In [4]:
%pip install --quiet "faiss-cpu>=1.7,<2" "ipywidgets>=7,<8" "pypdf>=3.8,<4"

Note: you may need to restart the kernel to use updated packages.


In [5]:
import json
import boto3
from pprint import pprint
from termcolor import colored
from utils import bedrock, print_ww
from utils.bedrock import bedrock_info

# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
# os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

print(colored("\n== FM lists ==", "green"))
pprint(bedrock_info.get_list_fm_models())

Create new client
  Using region: us-east-1
  Using profile: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)

== FM lists ==
{'Claude-Instant-V1': 'anthropic.claude-instant-v1',
 'Claude-V1': 'anthropic.claude-v1',
 'Claude-V2': 'anthropic.claude-v2',
 'Command': 'cohere.command-text-v14',
 'Jurassic-2-Mid': 'ai21.j2-mid-v1',
 'Jurassic-2-Ultra': 'ai21.j2-ultra-v1',
 'Titan-Embeddings-G1': 'amazon.titan-embed-text-v1',
 'Titan-Text-G1': 'TBD'}


## 랭체인 구성

LLM과 Embeddings 모델을 인스턴스화하는 것부터 시작합니다. 여기서는 텍스트 생성을 위해 Anthropic Claude를 사용하고 텍스트 삽입을 위해 Amazon Titan을 사용합니다.

참고: Bedrock과 함께 사용 가능한 다른 모델을 선택할 수도 있습니다. 모델을 변경하려면 `model_id`를 다음과 같이 교체하면 됩니다.

`llm = Bedrock(model_id="amazon.titan-tg1-large")`

사용 가능한 모델 ID는 다음과 같습니다.

- amazon.titan-tg1-large
- ai21.j2-grande-instruct
- ai21.j2-jumbo-instruct
- anthropic.claude-instant-v1
- anthropic.claude-v1

# 2. Titan Embedding 및 LLM 인 Claude-v2 모델 로딩

In [6]:
# We will be using the Titan Embeddings Model to generate our Embeddings.
from langchain.embeddings import BedrockEmbeddings
from langchain.llms.bedrock import Bedrock

# - create the Anthropic Model
llm = Bedrock(model_id="anthropic.claude-v2", client=boto3_bedrock, 
              model_kwargs={'max_tokens_to_sample':1000})
bedrock_embeddings = BedrockEmbeddings(client=boto3_bedrock)

In [7]:
llm, bedrock_embeddings

(Bedrock(client=<botocore.client.BedrockRuntime object at 0x7f582137f6d0>, model_id='anthropic.claude-v2', model_kwargs={'max_tokens_to_sample': 1000}),
 BedrockEmbeddings(client=<botocore.client.BedrockRuntime object at 0x7f582137f6d0>, region_name=None, credentials_profile_name=None, model_id='amazon.titan-embed-text-v1', model_kwargs=None, endpoint_url=None))

# 3. 데이터 준비
먼저 문서 저장소를 구축하기 위해 이미 제공된 "Samsung Knox Whitepaper" 문서를 사용하겠습니다.

[LangChain에서 사용 가능한 PyPDF의 DirectoryLoader](https://python.langchain.com/en/latest/reference/modules/document_loaders.html)를 사용하여 문서를 로드하고 더 작은 덩어리(chunk)로 나눌 수 있습니다.

참고: 검색된 문서/텍스트는 질문에 대답하기에 충분한 정보를 포함할 만큼 커야 합니다. 하지만 LLM 프롬프트에 들어갈 만큼 충분히 작습니다. 또한 임베딩 모델에는 입력 토큰 길이가 512개 토큰으로 제한되어 있으며 이는 한국어의 경우에 대략 180 ~ 200 글자로 변환됩니다. 이 사용 사례를 위해 [RecursiveCharacterTextSplitter](https://python.langchain.com/en/latest/modules/indexes/text_splitters/examples/recursive_text_splitter.html)를 사용하여 210자가 겹치는 약 50자의 청크를 생성합니다.

In [8]:
import numpy as np
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, PyPDFDirectoryLoader

loader = PyPDFDirectoryLoader("./data_knox/")

documents = loader.load()
# - in our testing Character split works better with this PDF data set
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 200,
    chunk_overlap  = 50,
)
docs = text_splitter.split_documents(documents)

In [9]:
avg_doc_length = lambda documents: sum([len(doc.page_content) for doc in documents])//len(documents)
avg_char_count_pre = avg_doc_length(documents)
avg_char_count_post = avg_doc_length(docs)
print(f'Average length among {len(documents)} documents loaded is {avg_char_count_pre} characters.')
print(f'After the split we have {len(docs)} documents more than the original {len(documents)}.')
print(f'Average length among {len(docs)} documents (after split) is {avg_char_count_post} characters.')

Average length among 107 documents loaded is 938 characters.
After the split we have 657 documents more than the original 107.
Average length among 657 documents (after split) is 156 characters.


In [10]:
print("docs[0].page_content: \n", docs[0].page_content)

docs[0].page_content: 
 Whitepaper:
Samsung KnoxTM
Security Solution
Version 2.2 May, 2017
Samsung  Research America
Samsung Electronics Co., Ltd.


In [11]:
sample_embedding = np.array(bedrock_embeddings.embed_query(docs[0].page_content))
print("Sample embedding of a document chunk: ", sample_embedding)
print("Size of the embedding: ", sample_embedding.shape)

Sample embedding of a document chunk:  [ 0.7734375  -0.296875   -0.27734375 ...  0.03955078 -0.20117188
 -0.51953125]
Size of the embedding:  (1536,)


유사한 패턴에 따라 전체 코퍼스에 대해 임베딩을 생성하고 벡터 저장소에 저장할 수 있습니다.

이는 임베딩 모델과 문서를 입력받아 전체 벡터 저장소를 생성하는 LangChain 내부의 [FAISS](https://github.com/facebookresearch/faiss)  구현을 사용하여 쉽게 수행할 수 있습니다. [LangChain](https://python.langchain.com/en/latest/modules/indexes/vectorstores)를 사용하면 프롬프트 생성, 쿼리 임베딩 가져오기, 관련 문서 샘플링 및 LLM 호출과 같은 대부분의 무거운 작업을 추상화할 수 있습니다. [VectorStoreIndexWrapper](https://python.langchain.com/docs/modules/data_connection/retrievers)가 이를 도와줍니다.



# 4.FAISS 벡터 스토어 생성

**⚠️⚠️⚠️ 참고: 다음 셀을 실행하는 데 몇 분 정도 걸릴 수 있습니다 ⚠️⚠️⚠️**

In [12]:
%%time

from langchain.chains.question_answering import load_qa_chain
from langchain.vectorstores import FAISS
from langchain.indexes import VectorstoreIndexCreator
from langchain.indexes.vectorstore import VectorStoreIndexWrapper

vectorstore_faiss = FAISS.from_documents(
    docs,
    bedrock_embeddings,
)

wrapper_store_faiss = VectorStoreIndexWrapper(vectorstore=vectorstore_faiss)

CPU times: user 1.76 s, sys: 79.2 ms, total: 1.84 s
Wall time: 59.5 s


In [13]:
wrapper_store_faiss

VectorStoreIndexWrapper(vectorstore=<langchain.vectorstores.faiss.FAISS object at 0x7f581a8db880>)

# 5.질문 및 답변

이제 벡터 저장소가 준비되었으므로 질문을 시작할 수 있습니다.

In [14]:
query = "\n\nHuman:What's Samsung Knox guard?\n\nAssistant:"

첫 번째 단계는 문서와 비교할 수 있도록 쿼리 임베딩을 만드는 것입니다.

In [15]:
query_embedding = vectorstore_faiss.embedding_function(query)
np.array(query_embedding)

array([-0.06884766, -0.46484375,  0.32421875, ...,  0.375     ,
        0.34375   , -0.234375  ])

이 쿼리 임베딩을 사용하여 관련 문서를 가져올 수 있습니다.
이제 쿼리는 임베딩으로 표시되어 가장 관련성이 높은 정보를 제공하는 데이터 저장소에 대해 쿼리의 유사성 검색을 수행할 수 있습니다.

In [16]:
relevant_documents = vectorstore_faiss.similarity_search_by_vector(query_embedding)
print(f'{len(relevant_documents)} documents are fetched which are relevant to the query.')
print('----')
for i, rel_doc in enumerate(relevant_documents):
    print_ww(f'## Document {i+1}: {rel_doc.page_content}')
    print('---')

4 documents are fetched which are relevant to the query.
----
## Document 1: For more information about Samsung Knox, visit  www.samsungknox.com
---
## Document 2: Samsung Knox Security Solution_V2.0 Nov. 17, 2016
Samsung Knox Security Solution_V1.12 September 22, 2016Copyright © 2017 Samsung Electronics Co. Ltd.
All rights reserved. Samsung is a registered
---
## Document 3: space.
Samsung Knox design
Knox addresses the most pressing security problems facing enterprises’
COPE and COBO strategies today. Samsung has identi ö ed the following key
---
## Document 4: In 2013, Samsung released Samsung Knox (TM) for any size enterprise,
aﬀ  ording them complete control over how they implement their mobile
security model.
---


이제 관련 문서가 준비되었으므로 LLM을 사용하여 이러한 문서를 기반으로 답변을 생성할 차례입니다.

유사성 검색 결과를 기반으로 검색된 관련 문서와 함께 초기 프롬프트를 사용합니다. 그런 다음 이를 결합하여 모델에 피드백하여 결과를 얻는 프롬프트를 생성합니다.

LangChain은 이를 쉽게 수행할 수 있는 방법에 대한 추상화를 제공합니다.

# 6. 빠른 방법
Vector Store를 둘러싸서 LLM 입력을 받는 LangChain에서 제공하는 래퍼를 사용할 수 있습니다.
이 래퍼는 뒤에서 다음 단계를 수행합니다.
- 질문을 입력합니다.
- 질문 임베딩 생성
- 관련 문서 가져오기
- 프롬프트에 문서와 질문을 채워 넣습니다.
- 프롬프트로 모델을 호출하고 사람이 읽을 수 있는 방식으로 답변을 생성합니다.

In [17]:
query, llm

("\n\nHuman:What's Samsung Knox guard?\n\nAssistant:",
 Bedrock(client=<botocore.client.BedrockRuntime object at 0x7f582137f6d0>, model_id='anthropic.claude-v2', model_kwargs={'max_tokens_to_sample': 1000}))

In [18]:
answer = wrapper_store_faiss.query(question=query, llm=llm)
print_ww(answer)


- Samsung Knox Guard is a security feature that comes built into Samsung Knox-enabled devices. It
provides real-time protection against malware and other threats by:

- Monitoring device integrity - It checks for any unauthorized changes to the firmware or OS during
runtime. This helps detect malware or rooting attempts.

- Analyzing app behavior - It monitors all apps and processes for any malicious or suspicious
activity in real-time. This helps catch malware, spyware, phishing attacks etc.

- Sending security alerts - If any suspicious activity is detected, it sends out alerts so you're
notified and can take action.

In summary, Knox Guard provides continuous monitoring and protection against malware, rooting, and
other threats. It's a key component of the defense-grade security provided by Samsung Knox.


다른 질문을 해보죠

In [19]:
query_2 = "\n\nHuman:What's the Knox? \n\nAssistant:"

In [20]:
answer_2 = wrapper_store_faiss.query(question=query_2, llm=llm)
print_ww(answer_2)

 Based on the context provided, Samsung Knox is a security platform designed by Samsung to address
security issues facing enterprises that allow employees to use personal (COBO) or company-provided
(COPE) devices in the workplace.

Key points about Samsung Knox from the context:

- It aims to address key security issues for enterprises adopting COPE/COBO strategies.

- It provides security features like FIPS 140-2 certification and separation of work and personal
data spaces.

- It uses proven hardware security mechanisms and a two-step design process focused on building a
trusted environment.

- It is a product/platform developed by Samsung to secure their devices used in enterprise settings.

So in summary, Samsung Knox is Samsung's enterprise mobile security platform for their devices. It
provides various security features to protect enterprise data on COPE/COBO devices.


#### 설명에 없는 내용 질문

In [21]:
query_3 = "\n\nHuman:How much using cost of Samsung knox seurity application?n\nAssistant:"
answer_3 = wrapper_store_faiss.query(question=query_3, llm=llm)
print_ww(answer_3)

 Based on the context provided, Samsung Knox security enhancements for existing Android managed
profiles on Samsung devices do not require a license activation fee. The Knox security features are
updated via OTA updates on Samsung devices running Android. So there is no additional cost to use
the Knox security features on Samsung devices.


# 7.사용자 정의 가능한 옵션
위 시나리오에서는 질문에 대한 상황 인식 답변을 빠르고 쉽게 얻을 수 있는 방법을 탐색했습니다. 이제 문서를 가져오는 방법을 사용자 정의할 수 있는 [RetrievalQA](https://python.langchain.com/en/latest/modules/chains/index_examples/Vector_db_qa.html)의 도움으로 더 사용자 정의 가능한 옵션을 살펴보겠습니다. `chain_type` 매개변수를 사용하여 프롬프트에 추가해야 합니다. 또한 검색해야 하는 관련 문서 수를 제어하려면 아래 셀에서 'k' 매개변수를 변경하여 다른 출력을 확인하세요. 많은 시나리오에서 LLM이 답변을 생성하는 데 사용한 소스 문서가 무엇인지 알고 싶을 수 있습니다. LLM 프롬프트의 컨텍스트에 추가된 문서를 반환하는 `return_source_documents`를 사용하여 출력에서 ​​해당 문서를 가져올 수 있습니다. 'RetrievalQA'를 사용하면 모델에 특정한 사용자 정의 [프롬프트 템플릿](https://python.langchain.com/en/latest/modules/prompts/prompt_templates/getting_started.html)을 제공할 수도 있습니다.

참고: 이 예에서는 Amazon Bedrock에서 LLM으로 Anthropic Claude를 사용하고 있습니다. 이 특정 모델은 입력이 'Human:' 아래에 제공되고 모델이 'Assistant:' 다음에 출력을 생성하도록 요청되는 경우 가장 잘 수행됩니다. 아래 셀에는 LLM이 기본 상태를 유지하고 컨텍스트 외부에서 응답하지 않도록 프롬프트를 제어하는 ​​방법의 예가 나와 있습니다.

In [24]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

prompt_template = """\n\nHuman: 
Please summarize the whole thing.
{context}

Question: {question}
\n\nAssistant:"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore_faiss.as_retriever(
        search_type="similarity", search_kwargs={"k": 10}
    ),
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)

query = "\n\nHuman:I am a developer preparing to develop with Knox. Where should I start?\n\nAssistant:"
result = qa({"query": query})
print_ww(result['result'])

 Here are some suggestions on where to start when developing with Samsung Knox:

- Get familiar with the Knox architecture and its components like the Trusted Execution Environment
(TEE), Secure Boot, and Real-time Kernel Protection. Understanding the foundations of Knox will help
you design secure apps.

- Review the Knox Standard SDKs and APIs available for developers. These include things like the
Knox Manageability API, Knox Attestation API, and more. Knowing what Knox APIs are available will
help you figure out what features you can build with Knox.

- Check the Knox Developer documentation and guides on the Samsung Developers site. There are code
samples, API references, and best practices to follow. Going through these will help you learn how
to properly integrate Knox into your apps.

- Look at the use cases and solution guides in the Knox section of the site. See examples of secure
authentication, data protection, application security, and more. These demonstrate how Knox is u

In [23]:
def show_context_used(context_list):
    for idx, context in enumerate(context_list):
        print("-----------------------------------------------")                
        print(f"{idx+1}. context to be fed into FM(e.g. Claude-v2)")
        print("-----------------------------------------------")        
        print_ww(context.page_content)        
        
show_context_used(result['source_documents'])        

-----------------------------------------------
1. context to be fed into FM(e.g. Claude-v2)
-----------------------------------------------
ready
The next several sections describe the technologies constructed in the
trusted environment to provide Knox enterprise optimizations.
SE for Android
Samsung Knox adopts the  (SE for
-----------------------------------------------
2. context to be fed into FM(e.g. Claude-v2)
-----------------------------------------------
The Samsung Knox philosophy       9
 Samsung Knox design        10
 Problem: Lack of trust in Android security     13
 Solution: Base security in a hardware-rooted trusted environment  13
-----------------------------------------------
3. context to be fed into FM(e.g. Claude-v2)
-----------------------------------------------
along with additional policy language. Knox's pioneering of SE for Android
has led to its adoption into the  (AOSP). The
Knox   is built on top of SE for Android to define a protected
------------------

# 8.결론
RAG(검색 증강 생성)에 관한 이 모듈을 완료한 것을 축하합니다! 이는 대규모 언어 모델의 성능과 검색 방법의 정확성을 결합하는 중요한 기술입니다. 검색된 관련 사례로 생성을 강화함으로써 우리가 받은 응답은 더욱 일관되고 근거가 있게 됩니다. 당신은 이 혁신적인 접근 방식을 배우는 것에 자부심을 느껴야 합니다. 나는 당신이 얻은 지식이 창의적이고 매력적인 언어 생성 시스템을 구축하는 데 매우 유용할 것이라고 확신합니다. 잘하셨어요!

위의 RAG 기반 질문 응답 구현에서 우리는 다음 개념과 Amazon Bedrock 및 LangChain 통합을 사용하여 이를 구현하는 방법을 탐색했습니다.

- 문서 로드 및 임베딩 생성을 통해 벡터 저장소 생성
- 질문에 대한 문서 검색
- LLM에 대한 입력으로 사용되는 프롬프트 준비
- 인간 친화적인 방식으로 답변 제시

### 요약
- 다양한 벡터 스토어로 실험해 보세요.
- Amazon Bedrock에서 사용 가능한 다양한 모델을 활용하여 대체 출력을 확인하세요.
- 임베딩 및 문서 청크의 영구 저장과 같은 옵션 탐색
- 엔터프라이즈 데이터 저장소와의 통합

